# ColabGCNdesign_beta_v0
This notebook is for running Dr. Shintaro Minami's [GCNdesign](https://github.com/ShintaroMinami/GCNdesign) on Google Colaboratory.

In [ ]:
#@title Install software
#@markdown Please run this cell by pressing the _Play_ ▶ button on the left.

#@markdown PyRosetta and GCNdesign will be installed.

#@markdown PyRosetta is ~1.5 GB. It takes several tens of minutes to install PyRosetta.

#@markdown  Enter your user ID and password to download PyRosetta.
#@markdown  You can obtain a license for PyRosetta at the following link https://els2.comotion.uw.edu/product/pyrosetta

import os, sys, time
user = "" #@param {type:"string"}
password = "" #@param {type:"string"}
!mkdir /content/PyRosetta
!cd /content/PyRosetta && wget -O _PyRosetta_.tar.bz2 --user $user --password $password http://graylab.jhu.edu/download/PyRosetta4/archive/release/PyRosetta4.MinSizeRel.python37.linux/PyRosetta4.MinSizeRel.python37.linux.release-316.tar.bz2
!tar xjf /content/PyRosetta/_PyRosetta_.tar.bz2 -C /content/PyRosetta --strip-components=1
pyrosetta_distr_path = '/content' + '/PyRosetta/_PyRosetta_.tar.bz2'
!cd /content/PyRosetta/setup && python setup.py install

for e in os.listdir('/content/PyRosetta/'):
    sys.path.append('/content/PyRosetta/' + '/' + e)

!pip install gcndesign

In [ ]:
#! /usr/bin/env python

import os
from os import path
import sys
import argparse
from google.colab import files

# argument parser
def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  else:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"

#@title Select a PDB file, enter options, and then press _Play_ ▶ button to design.

pdbcode = "" #@param {type:"string"}
pdb = get_pdb(pdbcode)
#@markdown - `pdbcode` Enter the PDB ID of the protein structure you want to design. ( e.g. 1qys )
#@markdown - **If you want to design your own structure, run the cell without writing anything in this field and then select the file.**
nstruct = 1 #@param {type:"integer"}
#@markdown - `nstruct` Number of structures output ( e.g. 10 )
prefix = "autodes3" #@param {type:"string"}
#@markdown - `prefix` Prefix for output PDB files. ( e.g. autodes ) 
prob_cut = 0.8 #@param {type:"number"}
#@markdown - `prob_cut` Probability cutoff. ( e.g. 0.8 )
scorefxn = "ref2015" #@param {type:"string"}
#@markdown - `scorefxn` Rosetta score function. ( e.g. ref2015 )
keep = ["1-10", "15", "30", "32-40"] #@param {type:"raw"}
#@markdown - `keep` Residue numbers for keeping the initial amino-acid type. ( e.g. [] or ["1-10","15","30","32-40"])
unused =  ["C", "H", "W"] #@param {type:"raw"}
#@markdown - `unused` Residue types not to be used in design sequences. ( e.g. None or["C",  "H", "W"] )
include_init_restype = False #@param {type:"boolean"}
#@markdown - `include_init_restype` Include the initial residue type.
fastdesign_iterations = 1 #@param {type:"integer"}
#@markdown - `fastdesign_iterations` "standard_repeats" for Rosetta FastDesign. ( e.g 10 )
paramin = None #@param {type:"raw"}
#@markdown - `paramin` NN parameter file. ( e.g. default: None )

# pyrosetta
import pyrosetta
pyrosetta.init("-ignore_unrecognized_res 1 -ex1 -ex2aro -detect_disulf 0")
scorefxn = pyrosetta.create_score_function(scorefxn)

# gcndesign predictor
#dir_script = path.dirname(path.realpath(__file__))
#sys.path.append(dir_script+'/../')
from gcndesign.predictor import Predictor
predictor = Predictor(param=paramin)

# pdb input
pose_in = pyrosetta.pose_from_pdb(pdb)

## Setup TaskFactory
taskf = pyrosetta.rosetta.core.pack.task.TaskFactory()
taskf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
if include_init_restype:
    taskf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())

# resfile task-operation
from gcndesign.resfile import fix_native_resfile, expand_nums
resfile = predictor.make_resfile(pdb=pdb, prob_cut=prob_cut, unused=unused)
resfile = fix_native_resfile(resfile, resnums=expand_nums(keep))
readresfile = pyrosetta.rosetta.core.pack.task.operation.ReadResfile()
readresfile.set_cached_resfile(resfile)

# add readresfile to taskfactory
taskf.push_back(readresfile)

## Check TaskFactory Setting
packer_task = taskf.create_task_and_apply_taskoperations(pose_in)

## Setup MoveMapFactory
movemapf = pyrosetta.rosetta.core.select.movemap.MoveMapFactory()
movemapf.all_bb(setting=True)
movemapf.all_chi(setting=True)
movemapf.all_jumps(setting=False)

## Check Setting
#display_pose = pyrosetta.rosetta.protocols.fold_from_loops.movers.DisplayPoseLabelsMover()
#display_pose.tasks(taskf)
#display_pose.movemap_factory(movemapf)
#display_pose.apply(pose)

## Mover Setting
fastdesign = pyrosetta.rosetta.protocols.denovo_design.movers.FastDesign(scorefxn_in=scorefxn, standard_repeats=fastdesign_iterations)
fastdesign.set_task_factory(taskf)
fastdesign.set_movemap_factory(movemapf)

## Apply
for i in range(nstruct):
    pose = pose_in.clone()
    fastdesign.apply(pose)
    file_out = '{:s}-{:03d}.pdb'.format(prefix, i+1)
    pose.dump_pdb(file_out)


# License
This notebook uses the GCNdesign source code and its parameters licensed under [MIT](https://github.com/ShintaroMinami/GCNdesign/blob/master/LICENSE).